In [1]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

print('bucket: ', bucket)
print('role: ', role)

No handlers could be found for logger "sagemaker"


('bucket: ', 'sagemaker-us-east-2-368560229227')
('role: ', u'arn:aws:iam::368560229227:role/service-role/AmazonSageMaker-ExecutionRole-20200831T184128')


In [3]:
pytorch_model = PyTorchModel(model_data='s3://sagemaker-us-east-2-368560229227/model.tar.gz', role=role, entry_point='inference.py', framework_version='1.3.1')

In [ ]:
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)

---

In [37]:
predictor

In [39]:
import boto3
import pandas as pd

bucket = 'sagemaker-us-east-2-368560229227'
data_key_cam1 = '2d_points_0_to_1_cam1_test_with_shift.csv'
data_key_cam2 = '2d_points_0_to_1_cam2_test_with_shift.csv'
data_location_cam1 = 's3://{}/{}'.format(bucket, data_key_cam1)
data_location_cam2 = 's3://{}/{}'.format(bucket, data_key_cam2)

df_cam1 = pd.read_csv(data_location_cam1)
df_cam2 = pd.read_csv(data_location_cam2)

In [51]:
import numpy as np
import torch
import json

cols = ['Sphere_X', 'Sphere_Y', 'Cube_X', 'Cube_Y']
cam1_sphere_cube_points = np.stack([df_cam1[col].values for col in cols], 1)
cam1_sphere_cube_points = torch.FloatTensor(cam1_sphere_cube_points)
cam2_sphere_cube_points = np.stack([df_cam2[col].values for col in cols], 1)
cam2_sphere_cube_points = torch.FloatTensor(cam2_sphere_cube_points)

time_offset = 1
cam1_clip_start_index = 67

clip_cam1 = cam1_sphere_cube_points[cam1_clip_start_index: (cam1_clip_start_index + 10)]
clip_cam2 = cam2_sphere_cube_points[(cam1_clip_start_index + time_offset) : (cam1_clip_start_index + 10 + time_offset)]

input_json = json.dumps({"clip_cam1": clip_cam1.numpy().tolist(), "clip_cam2": clip_cam2.numpy().tolist()})
# type(input_json)

In [48]:
# input_data = json.loads(input_json)
# clip_cam1 = input_data['clip_cam1']
# clip_cam2 = input_data['clip_cam2']

# clip_cam1

In [52]:
import boto3 
client = boto3.client('runtime.sagemaker')
 
response = client.invoke_endpoint(EndpointName='pytorch-inference-2020-09-04-03-55-56-385',
                                  Body=input_json)
response_body = response['Body'] 
print(response_body.read())

{"predicted time offset": 6}


In [47]:
# response = predictor.predict(input_json)
# print(response)

In [35]:
predictor.delete_endpoint()